In [ ]:
#Identity gate 
def IDTY(a):
  if a == 0:
    return 0
  else:
    return 1

if __name__ == '__main__':
  print("The output of input 0 is: \n", IDTY(0))
  print("The output of input 1 is: \n", IDTY(1))

In [ ]:
#NOT gate
def NOT(a):
  if a == 0:
    return 1
  else:
    return 0

if __name__ == '__main__':
  print("The output of NOT gate for input 0 is: \n", NOT(0))
  print("The output of NOT gate for input 1 is: \n", NOT(1))

In [ ]:
#OR gate
def OR(a,b):
  if a == 0 and b == 0:
    return 0
  else:
    return 1

if __name__ == '__main__':
  print("The output of OR gate for inputs 0,0 is: \n", OR(0,0))
  print("The output of OR gate for inputs 0,1 is: \n", OR(0,1))
  print("The output of OR gate for inputs 1,0 is: \n", OR(1,0))
  print("The output of OR gate for inputs 1,1 is: \n", OR(1,1))

In [ ]:
#AND gate 
def AND(a,b):
  if a == 1 and b == 1:
    return 1
  else:
    return 0

if __name__ == '__main__':
  print("The output of AND gate for inputs 0,0 is: \n", AND(0,0))
  print("The output of AND gate for inputs 0,1 is: \n", AND(0,1))
  print("The output of AND gate for inputs 1,0 is: \n", AND(1,0))
  print("The output of AND gate for inputs 1,1 is: \n", AND(1,1))

In [ ]:
#XOR gate
def XOR(a,b):
  if a != b:
    return 1
  else:
    return 0

if __name__ == '__main__':
  print("The output of XOR gate for inputs 0,0 is: \n", XOR(0,0))
  print("The output of XOR gate for inputs 0,1 is: \n", XOR(0,1))
  print("The output of XOR gate for inputs 1,0 is: \n", XOR(1,0))
  print("The output of XOR gate for inputs 1,1 is: \n", XOR(1,1))

In [ ]:
#NOR gate
def NOR(a,b):
  if a == 0 and b == 0:
    return 1
  else:
    return 0

if __name__ == '__main__':
  print("The output of NOR gate for inputs 0,0 is: \n", NOR(0,0))
  print("The output of NOR gate for inputs 0,1 is: \n", NOR(0,1))
  print("The output of NOR gate for inputs 1,0 is: \n", NOR(1,0))
  print("The output of NOR gate for inputs 1,1 is: \n", NOR(1,1))

In [ ]:
#Import packages required
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42) #for reproducibility

# Define the Quantum and Classical Registers
qr = QuantumRegister(1)
cr = ClassicalRegister(1)

# Build the circuit
sup = QuantumCircuit(qr, cr)
sup.x(qr) #invert the qubit (qubits are initialized to |0>, so this turns |0>. to |1>)
sup.h(qr)
sup.measure(qr, cr)
sup.draw(output='mpl')
plt.show()

# Execute the circuit (using the qasm simulator)
job = execute(sup, backend = Aer.get_backend('qasm_simulator'), shots=1024)
result = job.result()

# Print the result
print(result.get_counts(sup))

#plot the results
counts = result.get_counts(sup)
plot_histogram(counts)
plt.show()

In [ ]:
#QECC Code

from qiskit import *
from qiskit.ignis.verification.topological_codes import RepetitionCode
from qiskit.ignis.verification.topological_codes import GraphDecoder
from qiskit.ignis.verification.topological_codes import lookuptable_decoding, postselection_decoding
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager
from qiskit import QuantumCircuit, execute, Aer
from qiskit.providers.aer import noise
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit import QuantumRegister, ClassicalRegister
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42) #for reproducibility

def get_noise(p_meas,p_gate):

	error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
	error_gate1 = depolarizing_error(p_gate, 1)
	error_gate2 = error_gate1.tensor(error_gate1)

	noise_model = NoiseModel()
	noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
	noise_model.add_all_qubit_quantum_error(error_gate1, ["x"]) # single qubit gate error is applied to x gates
	noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
   	 
	return noise_model
    
noise_model = get_noise(0.05,0.05)

qc0 = QuantumCircuit(3,3,name='0') # initialize circuit with three qubits in the 0 state

qc0.measure(qc0.qregs[0],qc0.cregs[0]) # measure the qubits

# run the circuit with th noise model and extract the counts
counts = execute( qc0, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()

print(counts)

qc1 = QuantumCircuit(3,3,name='0') # initialize circuit with three qubits in the 0 state
qc1.x(qc1.qregs[0]) # flip each 0 to 1

qc1.measure(qc1.qregs[0],qc1.cregs[0]) # measure the qubits

# run the circuit with th noise model and extract the counts
counts = execute( qc1, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()

print(counts)

noise_model = get_noise(0.5,0.0)
counts = execute( qc1, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()
print(counts)

cq = QuantumRegister(2,'code\ qubit\ ')
lq = QuantumRegister(1,'auxiliary\ qubit\ ')
sb = ClassicalRegister(1,'syndrome\ bit\ ')
qc = QuantumCircuit(cq,lq,sb)
qc.cx(cq[0],lq[0])
qc.cx(cq[1],lq[0])
qc.measure(lq,sb)
qc.draw(output='mpl')

qc_init = QuantumCircuit(cq)
(qc_init+qc).draw(output='mpl')

counts = execute( qc_init+qc, Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:',counts)

qc_init = QuantumCircuit(cq)
qc_init.x(cq)
(qc_init+qc).draw(output='mpl')

counts = execute( qc_init+qc, Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:',counts)

qc_init = QuantumCircuit(cq)
qc_init.h(cq[0])
qc_init.cx(cq[0],cq[1])
(qc_init+qc).draw(output='mpl')

counts = execute( qc_init+qc, Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:',counts)

code = RepetitionCode(3,1) #3 qubit, one round of syndrome measurement

for reg in code.circuit['0'].qregs+code.circuit['1'].cregs:
	reg.name = reg.name.replace('_','\ ') + '\ '

code.circuit['0'].draw(output='mpl')
code.circuit['1'].draw(output='mpl')

def get_raw_results(code,noise_model=None):

    circuits = code.get_circuit_list()
    raw_results = {}
    for log in range(2):
        job = execute( circuits[log], Aer.get_backend('qasm_simulator'), noise_model=noise_model)
        raw_results[str(log)] = job.result().get_counts(str(log))
    return raw_results

raw_results = get_raw_results(code)
for log in raw_results:
    print('Logical',log,':',raw_results[log],'\n')

    
code = RepetitionCode(3,1)

noise_model = get_noise(0.05,0.05)

raw_results = get_raw_results(code,noise_model)
for log in raw_results:
	print('Logical',log,':',raw_results[log],'\n')
    
circuits = code.get_circuit_list()
table_results = {}
for log in range(2):
	job = execute( circuits[log], Aer.get_backend('qasm_simulator'), noise_model=noise_model, shots=10000 )
	table_results[str(log)] = job.result().get_counts(str(log))
    
P = lookuptable_decoding(raw_results,table_results)
print('P =',P)
plt.show()

In [ ]:
#superdence coding

 #Import modules
from qiskit import *
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42) #for reproducibility

#Create the EPR pair for both Alice (a) and Bob (b) for the QuantumCircuit (qc)
def create_bell_pair(qc, a, b):
	qc.h(a)
	qc.cx(a,b)
    
#Encode the message (classical bits) to be sent, on Alice's side. The classical bits are '00', '01', '10', or '11'
def encode_message(qc, qubit, msg):
    if msg == "00":
        pass    
    elif msg == "10":
        qc.x(qubit) 
    elif msg == "01":
        qc.z(qubit) 
    elif msg == "11":
        qc.z(qubit) 
        qc.x(qubit) 
    else:
        print("Invalid Message: Sending '00'")

   	 
#Decode the message on Bob's side
def decode_message(qc, a, b):
	qc.cx(a,b)
	qc.h(a)
    
#Implement the superdense coding protocol
qc = QuantumCircuit(2)

create_bell_pair(qc, 0, 1)
qc.barrier() # This is the barrier (used to improve clarity on the circuit)

message = "11" #Alice applies Z gate followed by X gate
encode_message(qc, 0, message)
qc.barrier()

decode_message(qc, 0, 1)

qc.measure_all()

qc.draw(output = "mpl")
#plt.show()

#Run the simulation using IBM classical simulator. Then visualize the results using the histogram
backend = Aer.get_backend('qasm_simulator')
job_sim = execute(qc, backend, shots=1024)
sim_result = job_sim.result()

measurement_result = sim_result.get_counts(qc)
print(measurement_result)
plot_histogram(measurement_result)
plt.show()